In [7]:
import pymongo
import servidor_mongo as SM
import json
from bson.objectid import ObjectId
from datetime import datetime, timedelta
import utils
import time

import tensorflow as tf
# Importamos la librería de pandas
import pandas as pd
import re
import matplotlib as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import pandas as pd

## Nota: Una variable importante que de no estar almacenada debe darse de igual manera es la distancia entre la antena y el tag -> debería ser un dato almacenado

## Debe estar en centimetros 158
#D_ant_tag = 177 

## Leemos el json con los datos de las antenas y el tag
f = open("Ant_tag.json", "r")
c = f.read()
f.close()
js = json.loads(c)

## Establecemos la conexion con el servidor
base_de_datos = SM.obtener_bd()

## Traemos el ultimo elemento de la lista en el tiempo de lectura
## La idea es poder trabajar con el tiempo anterior a este número
r_0 = base_de_datos.find().sort("_id",-1).limit(1)

for r in r_0:
    id_0 = r

respuestas = base_de_datos.find().sort("_id",-1)#.limit(500)

# tag_1/Ant_1
vect_11 = []
# tag_1/Ant_2
vect_12 = []

for r in respuestas:
    if(r['Id_tag']== js["tag_1"] and r['Id_Ant']==js["Antena_1"]):
        vect_11.append(r)
    elif(r['Id_tag']== js["tag_1"] and r['Id_Ant']==js["Antena_2"]):
        vect_12.append(r)
    else:
        pass
## De los cuatro vector que creamos, los reducimos a dos para poder tener las variables que tienen el mismo tag con antenas diferentes
## El críterio para juntar alguno de los elementos es que tengan la misma hora de captura del dato y que el tag sea igual en ambos casos pero los datos deben ser tomado de diferentes antenas
## En este caso tenemos solo dos antenas y dos tag

vect_1 = []
vect_2 = []
# cargamos los datos al vect_1
for r in vect_11:
    for s in vect_12:
        if(r['hora']==s['hora']):
            #print("3")
            vect_1.append(r)
            vect_1.append(s)
            vect_12.remove(s)

## Para ingresar a los elementos del arreglo y modificarlos
## Actualizo los datos añadiendo el parametro Distancia_ant_tag para guardarlos en la base de datos de MongoDb
## Vector 1
for t in range(int(len(vect_1)/2)):
    D_B,D_A = utils.Distancia_ant_tag(vect_1[2*t]['Distancia_entre_ant'],vect_1[2*t]['Ang_azimuth'],vect_1[(2*t)+1]['Ang_azimuth'])
    d_a = D_A
    d_b = D_B
    base_de_datos.update_one(
        {"_id": ObjectId(str(vect_1[2*t]['_id']))},
        {
            "$set":{
                "Distancia_ant_tag": d_b
            }
        }
    )
    base_de_datos.update_one(
        {"_id": ObjectId(vect_1[(2*t)+1]['_id'])},
        {
            "$set":{
                "Distancia_ant_tag": d_a
            }
        }
    )